# Definindo os agentes Codificador e Revisor

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class LLMAgent:
    def __init__(self, model_name="gpt2", agent_type="codificador"):
        """
        Inicializa o agente LLM com o modelo GPT-2.

        Parameters:
        model_name (str): Nome do modelo a ser utilizado (GPT-2 neste exemplo).
        agent_type (str): O tipo do agente ("codificador" ou "revisor").
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.agent_type = agent_type
        self.history = []  # Histórico de interações entre os agentes

        # Definir o pad_token_id explicitamente
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def generate_code(self, prompt):
        """
        Gera um código ou feedback baseado no prompt fornecido.

        Parameters:
        prompt (str): O enunciado do problema ou o código a ser revisado.

        Returns:
        str: O código gerado pelo agente.
        """
        # Tokenizar o prompt com attention_mask
        inputs = self.tokenizer.encode_plus(
            prompt,
            return_tensors="pt",  # Retorna tensores do PyTorch
            padding=True,  # Adiciona padding automaticamente
            truncation=True,  # Corta se o prompt for muito longo
            max_length=512  # Limita o comprimento dos tokens se necessário
        )

        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        # Geração com pad_token_id e attention_mask definidos
        outputs = self.model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=100,
            num_return_sequences=1,
            pad_token_id=self.tokenizer.pad_token_id  # Evita o warning do pad_token_id
        )

        #outputs = self.model.generate(inputs, max_length=150, num_return_sequences=1)
        #outputs = self.model.generate(inputs, max_new_tokens=100, num_return_sequences=1)

        result = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        self.history.append(result)  # Armazena a interação no histórico
        return result

    def review_code(self, code):
        """
        Revê o código gerado, sugerindo melhorias.

        Parameters:
        code (str): O código gerado pelo agente codificador.

        Returns:
        str: O feedback ou revisão do código.
        """
        review_prompt = f"Reveja o seguinte código e sugira melhorias:\n\n{code}"
        return self.generate_code(review_prompt)

# Exemplo de uso:
# codificador = LLMAgent(agent_type="codificador")
# revisor = LLMAgent(agent_type="revisor")


# Definindo o ambiente para os agentes interagirem

In [4]:
# Utilizaremos a biblioteca Gym para definir um ambiente onde os agentes possam interagir.
#  A função do ambiente é atribuir recompensas com base na qualidade do código e do feedback.

import gym
from gym import spaces
import numpy as np

class DataAnalysisEnv(gym.Env):
    """
    Um ambiente personalizado para treinar agentes de RL que resolvem problemas de análise de dados.
    """
    def __init__(self, dataset, expected_output, codificador, revisor):
        super(DataAnalysisEnv, self).__init__()

        # Armazena os agentes LLM
        self.codificador = codificador
        self.revisor = revisor

        # Definindo o espaço de ação e observação
        self.action_space = spaces.Discrete(3)  # Exemplo: (0 = continuar, 1 = revisar, 2 = finalizar)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)

        # Dados e solução esperada
        self.dataset = dataset
        self.expected_output = expected_output
        self.current_state = 0  # Representa o progresso do agente (inicializado como 0)
        self.max_steps = 10  # Número máximo de tentativas

        # Internamente, mantemos um histórico de interações
        self.steps_taken = 0
        self.done = False
        self.current_code = ""  # Código gerado pelos agentes

    def reset(self):
        """
        Reseta o ambiente para um novo episódio.
        """
        self.current_state = 0
        self.steps_taken = 0
        self.done = False
        self.current_code = ""  # Limpa o código gerado
        return np.array([self.current_state], dtype=np.float32)

    def step(self, action):
        """
        Executa uma ação no ambiente (o código gerado ou revisado pelos agentes).

        Parameters:
        action (int): A ação que o agente deseja executar.

        Returns:
        Tuple: A nova observação, recompensa, flag de término e informações adicionais.
        """
        self.steps_taken += 1
        reward = 0

        if action == 0:  # Continuar gerando código
            # Codificador gera mais código
            new_code = self.codificador.generate_code(f"Dados: {self.dataset}\n")
            self.current_code += new_code
            reward = -1  # Penalidade por não concluir a tarefa ainda

        elif action == 1:  # Revisar código
            # Revisor analisa o código e sugere melhorias
            feedback = self.revisor.review_code(self.current_code)
            reward = 1  # Recompensa por sugerir melhorias (simulação)

        elif action == 2:  # Finalizar o processo
            # Verifica se o código está correto
            if self._evaluate_solution():
                reward = 10  # Recompensa por gerar a solução correta
                self.done = True
            else:
                reward = -10  # Penalidade por finalizar incorretamente
                self.done = True

        # Atualiza o estado do ambiente
        self.current_state = np.random.rand()
        if self.steps_taken >= self.max_steps:
            self.done = True

        return np.array([self.current_state], dtype=np.float32), reward, self.done, {}

    def _evaluate_solution(self):
        """
        Função de avaliação que compara o código gerado com a saída esperada.
        """
        # Simulação de avaliação. Aqui você pode implementar uma função mais complexa
        # para avaliar se o código gerado está correto.
        return "visualização de dados" in self.current_code


# Codificador e Revisor no Ambiente: O ambiente recebe o codificador e o revisor como parâmetros e usa suas funções para gerar e revisar o código em cada passo.

# Ação do Codificador: Quando o agente decide continuar gerando código (action == 0), a função generate_code do Codificador é chamada e o código é atualizado no ambiente.

# Ação do Revisor: Quando o agente decide revisar o código (action == 1), o Revisor revisa o código gerado e sugere melhorias.

# Finalizar: Se o agente decide finalizar o processo (action == 2), a função de avaliação compara o código atual com a solução esperada e atribui uma recompensa dependendo do resultado.

# Treinamento dos agentes

In [ ]:
# Usaremos a biblioteca stable-baselines3 para realizar o treinamento dos agentes.
# Usaremos o algoritmo PPO (Proximal Policy Optimization), que é adequado para esse tipo de problema

from stable_baselines3 import PPO
import torch

# Verifica se a GPU está disponível e escolhe o dispositivo correto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_agents():
    """
    Treina os agentes codificador e revisor no ambiente de análise de dados.
    """
    # Instanciando os agentes LLM
    codificador = LLMAgent(agent_type="codificador")
    revisor = LLMAgent(agent_type="revisor")

    # Instanciando o ambiente de RL
    dataset = "Walmart.csv"
    #expected_output = "Solução correta de visualização de dados"
    expected_output = {
    "type": "visualization",
    "description": "Gráficos de linha mostrando as vendas ao longo do tempo, separados por categorias de produtos, com uma linha de tendência e intervalos de confiança.",
    "components": [
        {"chart_type": "line", "title": "Vendas ao Longo do Tempo"},
        {"chart_type": "bar", "title": "Vendas por Categoria"},
        {"chart_type": "heatmap", "title": "Análise de Sazonalidade"}
    ]
}
    env = DataAnalysisEnv(dataset, expected_output, codificador, revisor)

    # Configurando o modelo de aprendizado por reforço PPO para usar GPU, se disponível
    model = PPO("MlpPolicy", env, verbose=1, device=device)

    # Treinando o modelo por 10.000 passos
    model.learn(total_timesteps=10000)

    # Salvando o modelo treinado
    model.save("ppo_agents")

    # Carregar o modelo para uso posterior
    # model = PPO.load("ppo_agents")

# teste do resultado final
def test_trained_model():
    """
    Testa o modelo treinado em um novo episódio.
    """
    # Carrega o modelo treinado
    model = PPO.load("ppo_agents")

    # Instanciando os agentes
    codificador = LLMAgent(agent_type="codificador")
    revisor = LLMAgent(agent_type="revisor")

    # Instanciando o ambiente
    dataset = "sales_data.csv"
    expected_output = "Solução correta de visualização de dados"
    env = DataAnalysisEnv(dataset, expected_output, codificador, revisor)

    obs = env.reset()

    for _ in range(10):
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        print(f"Ação tomada: {action}, Recompensa: {reward}")

        if done:
            print("Episódio finalizado.")
            break

import time

if __name__ == "__main__":
    train_agents()

    # Teste o modelo treinado
    #test_trained_model()